Python поддерживает **наследование** (*inheritance*) это механизм, позволяющий создавать новый класс на основе существующего. Это позволяет строить иерархии классов – от общего к частному.

Класс, от которого наследуют, называется **родительским** (*parent class*), **базовым** (*base class*) или **суперклассом** (*superclass*). Класс-наследник называется **дочерним** (*child class*) или **подклассом** (*subclass*).

Подкласс наследует атрибуты суперкласса, но может их **переопределять** (*override*) и добавлять свои атрибуты. 

Создадим базовый класс:

In [ ]:
class Cell:
    """Базовый класс биологической клетки"""
    
    def __init__(self, size, membrane_type="липидный бислой"):
        self.size = size  # размер в микрометрах
        self.membrane_type = membrane_type
        self.is_alive = True
    
    def divide(self):
        """Деление клетки"""
        print(f"Клетка размером {self.size} мкм делится")
        new_size = self.size / 2
        self.size = new_size    # исходная клетка уменьшается
        return Cell(new_size, self.membrane_type)
    
    def get_info(self):
        """Информация о клетке"""
        return f"Клетка: размер={self.size} мкм, мембрана={self.membrane_type}"

## Расширение базового класса

Создадим класс эпителиальной клетки, который наследуется от `Cell`. Добавляется атрибут класса `self.layer_type` – тип слоя (`"simple squamous"` означает, что клетка является однослойной плоской). Также добавляется метод объекта `secrete()`. Таким образом класс `EpithelialCell` расширяет (*extend*) базовый класс `Cell`.

In [ ]:
class EpithelialCell(Cell):
    """Эпителиальная клетка — выстилает поверхности органов"""
    
    def __init__(self, size, layer_type="simple squamous"):
        super().__init__(size)
        self.layer_type = layer_type
    
    def secrete(self, substance):
        """Секреция веществ"""
        print(f"Эпителиальная клетка выделяет {substance}")

`super().__init__()` – вызов инициализатора родительского класса.

`Neuron` автоматически получает поля объектов `size`, `membrane_type`, `is_alive` и методы `divide()`, `get_info()`.

Может возникнуть закономерный вопрос: а как встроенная функция `super()` узнает, о каком классе идет речь, если мы не передали в него никой аргумент? Здесь работает «магия» компилятора Python 3. Когда Python компилирует метод класса, он автоматически создаёт скрытую переменную `__class__`, которая ссылается на класс, в котором метод определён.

In [132]:
class SubCell(Cell):
    def __init__(self):
        print(__class__)
        # super().__init__(15)
        super(__class__, self).__init__(15)
    
SubCell()

<class '__main__.SubCell'>


In [126]:
__class__

NameError: name '__class__' is not defined

In [ ]:
EpithelialCell.__dict__

Создадим экземпляр однослойной кубической (`"simple cuboidal"`) эпителиальной клетки:

In [ ]:
epithelial = EpithelialCell(15, "simple cuboidal")
epithelial.__dict__

In [ ]:
epithelial.get_info()

In [ ]:
e2 = epithelial.divide()
type(e2)

In [ ]:
e2.get_info()

In [ ]:
epithelial.get_info()

## Переопределение методов

Дочерний класс может переопределить методы родителя (*Method Overriding*):

In [ ]:
class RedBloodCell(Cell):
    """Эритроцит (красная кровяная клетка)"""
    
    def __init__(self, size=7):
        super().__init__(size)
        self.hemoglobin_count = 270_000_000  # молекул гемоглобина
    
    def get_info(self):
        """Переопределяем метод родителя"""
        base_info = super().get_info()  # получаем информацию от родителя
        return f"{base_info}, гемоглобин={self.hemoglobin_count} молекул"
    
    def divide(self):
        """Эритроциты не делятся!"""
        print("Эритроциты не способны к делению")
        return None

In [ ]:
erythrocyte = RedBloodCell()
erythrocyte.get_info()

In [ ]:
isinstance(erythrocyte, RedBloodCell)

In [ ]:
isinstance(erythrocyte, Cell)

In [ ]:
RedBloodCell.__bases__

In [ ]:
Cell.__bases__

Несмотря на то, что при определении класса `Cell` мы не задавали никаких классов для наследования, тем не менее, по умолчанию класс наследуется от базового класса `object`. Соответственно, в классе `Cell` содержатся не только поля и методы, которые мы в нем определили, но также поля и методы, которые определены в суперклассе `object`.

### Порядок разрешения методов (MRO)

Иногда в случае цепочки наследований сложно отследить в каком классе определен метод, вызываемый у объекта. Для выбора метода, который будет вызван у объект, класс которого может в общем случае наследовать методы с одним и тем же именем от разных классов, Python руководствуется определенным правилом – **порядком разрешения методов** (*Method Resolution Order*).

Рассмотрим абстрактный пример наследования:

In [ ]:
class A:
    def f(self): print("A.f")

class B:
    def f(self): print("B.f")

class C(A, B):
    pass

c = C()
c.f()

При вызове метода `f()` класс `C` использует метод, определенный в классе `A`, так как он шел первым аргументом при определении класса `C`.

Поле `__mro__` класса содержит последовательность наследования в виде кортежа. Метод класса `mro()` также возвращает последовательность наследования, но в виде списка. Посмотрим на порядок наследования в классе `C`:

In [ ]:
C.__mro__, C.mro()

Напишем функцию, которая для заданного объекта и имени метода возвращает класс, в котором определен метод:

In [ ]:
def findDefiningClass(obj, methodName):

    typeOfObject = type(obj)            # возвращает класс объекта
    for t in typeOfObject.mro():        # перебирается список иерархии классов;
        if methodName in t.__dict__:    # если имя метода обнаруживается в
            return t                    # словаре атрибутов класса t, возвращается t

In [ ]:
findDefiningClass(c, 'f')